In [4]:
%pip install opencv-python matplotlib -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
# from hand_transform import *
import torchvision.transforms.v2 as transforms

from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

import torch
from torchvision.transforms import v2
import torchvision.models as visionmodels

import torch.nn as nn

import lightning as L
import numpy as np
import cv2 as cv

from asl_transforms import RandomNoiseBackgroundTransform

In [2]:
img_size = 224

# See https://pytorch.org/vision/master/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py
# for more examples of transforms

# Open Idea: Grayscale for anti bias


data_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.3, saturation=0.3, hue=0.3), # Idea: ColorJitter for anti bias
    transforms.RandomRotation(degrees=5),
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet stats
])

/opt/conda/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [19]:
t = transforms.Compose(transforms=(
    transforms.Resize(size=(224, 224)),
    RandomNoiseBackgroundTransform(brightness=1.0, contrast=0.0, saturation=1.0, hue=0.5),
    transforms.RandomPerspective(),
    transforms.ColorJitter()
    #transforms.ColorJitter(brightness=1.0, contrast=0.0, saturation=1.0, hue=0.5),
    #transforms.Grayscale(),
    #transforms.GaussianBlur(kernel_size=(3,3),sigma=3)
))

In [20]:
# orig_img = Image.open(Path("/home/jovyan/Test-Hands/A") / 'A_1.jpeg')
orig_img = Image.open(Path(r"/exchange/dspro2/silent-speech/ASL_Pictures_Dataset/Train/D") / '10.jpg')
bg = r"/exchange/dspro2/silent-speech/ASL_Pictures_Dataset/Train/A/10.jpg"

(224, 224, 1)
(224, 224, 3)


In [28]:
plt.imshow(t(orig_img))

(224, 224, 1)
(224, 224, 3)
(224, 224, 3) uint8


In [14]:
mask = np.zeros((224,224), dtype=np.uint8)
img = np.ones((224,224,3), dtype=np.uint8)

noise = np.random.uniform(0, 255, size=img.shape).astype(np.uint8)

mask = np.expand_dims(mask, -1)
print(mask.shape)
print((img * mask / 255).shape)

img_with_noise = np.astype(img * (mask / 255) + noise * ((255 - mask) / 255), np.uint8)

img_with_noise = cv.GaussianBlur(img_with_noise, (3, 3), 1.5)
print(img_with_noise.shape, img_with_noise.dtype)

Image.fromarray(img_with_noise)